In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from skimage.restoration import denoise_bilateral
from skimage.transform import resize, rescale
from scipy import ndimage
from scipy import signal
from skimage.draw import circle_perimeter_aa



In [ ]:
IMDIR = r"C:\Users\maria\DataSim\ATRIM\TP2-ATRIM-Spatial Filtering\ATRIM-TP2\images"

In [ ]:
f_charlie = os.path.join(IMDIR, "charlie/charlie-template.jpeg")
f = os.path.join(IMDIR, "charlie/marche-crop.png")

In [ ]:
def normalize(im):
    im = (im-im.min())/(im.max()-im.min())
    return im

In [ ]:
#Display properties
width=15
height=5

# Read and preprocess image
charlie = io.imread(f_charlie, as_gray=True)
charlie = charlie.astype(float)
charlie = normalize(charlie)

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = normalize(im)

fig=plt.figure(figsize=(width, height))
plt.subplot(1,2,1)
plt.imshow(im, cmap = 'gray')
plt.title('im')
plt.subplot(1,2,2)
plt.imshow(charlie, cmap = 'gray')
plt.title('charlie')
plt.show()

In [ ]:
def NCC(imsource,kernel):

    # Find image and kernel sizes
    im_shape = imsource.shape
    imh,imw = im_shape[0], im_shape[1]
    kh,kw = kernel.shape
    print(kh)
    delta_h=int((kh-1)/2)
    delta_w=int((kw-1)/2)
    
    print(delta_h)
    # Image padding
    imPadded = np.zeros((imh+2*delta_h,imw+2*delta_w))
    imPadded[delta_h:imh+delta_h,delta_w:imw+delta_w] = imsource

    
    # Create an empty image to store the result
    imNCC = np.zeros((imh,imw))
    
    ####################################################################################
    #BEGIN FILL IN
    u_kernel = np.mean(kernel);
    std_kernel = np.std(kernel);
    
    ncc_max = -1
    x_max = -1
    y_max = -1
    
    for x in range(imh):
        for y in range(imw):
            
                imPatch = imPadded[x:x+kh, y:y+kw] #création du patch
                
                if imPatch.shape == kernel.shape:
                    u_patch = np.mean(imPatch)
                    std_patch = np.std(imPatch)

                    if std_patch > 0:
                        ncc = np.sum((imPatch - u_patch)*(kernel-u_kernel))/(std_patch*std_kernel)
                        
                        imNCC[x, y] = ncc
                        
                        if ncc>ncc_max: #trouve le ncc max
                            ncc_max = ncc
                            x_max = x
                            y_max =y

    return x_max, y_max #reenvoye les coordonés du ncc max
                    
#END FILL


In [ ]:
width=15
height=5

# Read and preprocess image
charlie = io.imread(f_charlie, as_gray=True)
charlie = charlie.astype(float)
charlie = normalize(charlie)

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = normalize(im)

x_max, y_max = NCC(im, charlie)

charlie_trouve = im.copy()
debut = y_max - int(charlie.shape[1]/2)
fin =  x_max - int(charlie.shape[0]/2)
# Dessiner un rectangle autour de l'emplacement de Charlie 
top_left = (debut, fin)  # Inverser pour obtenir (x, y)
bottom_right = (top_left[0] + charlie.shape[1], top_left[1] + charlie.shape[0])

# Définir l'épaisseur du rectangle
thickness = 3  # Épaisseur de 3 pixels

# Mettre à jour l'image pour marquer la position de Charlie
# Dessiner le rectangle en modifiant les pixels de l'image
for t in range(thickness):  # Pour chaque épaisseur
    # Ligne supérieure
    charlie_trouve[top_left[1] - t, top_left[0]:bottom_right[0]] = 0
    # Ligne inférieure
    charlie_trouve[bottom_right[1] - 1 + t, top_left[0]:bottom_right[0]] = 0
    # Colonne gauche
    charlie_trouve[top_left[1]:bottom_right[1], top_left[0] - t] = 0
    # Colonne droite
    charlie_trouve[top_left[1]:bottom_right[1], bottom_right[0] - 1 + t] = 0

# Afficher le résultat



fig=plt.figure(figsize=(width, height))
plt.subplot(1,3,1)
plt.imshow(im, cmap = 'gray')
plt.title('im')
plt.subplot(1,3,2)
plt.imshow(charlie, cmap = 'gray')
plt.title('charlie')
plt.subplot(1,3,3)
plt.imshow(charlie_trouve, cmap='gray')
plt.title('Position de Charlie trouvée')
plt.axis('off')  # Désactiver les axes
plt.axis('off')  # Désactiver les axes
plt.show()


plt.show()